In [1]:
import os
import math
import time
import argparse
from tqdm import tqdm

import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as ls
import torch.utils.data as d
from AttributeClassifier import AttributeClassifier, AttributeClassifierMobileNet
from LargeScaleAttributesDataset import LargeScaleAttributesDataset
from torchvision import transforms
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from scheduler import GradualWarmupScheduler
import pickle
import utils as u

In [2]:
dataset_root = '../../data/largescale/'
image_input_size = 299
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
test = pickle.load( open(  os.path.join(dataset_root,"test.pkl"), "rb" ) )
test.transform = transforms.Compose([ transforms.CenterCrop(image_input_size), transforms.ToTensor(), normalize ])

In [8]:
device = torch.device('cuda')
xception_model = AttributeClassifier(out_features=337, device=device)

final_checkpoint = torch.load('../trained_models/final_model.chkpt')
xception_model.load_state_dict(final_checkpoint['model'])
xception_model.eval()
print('Loaded')

Loaded


In [4]:
data = d.DataLoader(test, batch_size=32, shuffle=True, drop_last=False, num_workers=2, pin_memory=True)

In [5]:
def run_epoch(model, data, device, is_train=False, optimizer=None, calculate_every=100):
    if is_train: model.train()
    else: model.eval()

    total_loss = 0
    n_total = 0
    n_total_correct = 0
    iterations = 0
    total_f1 = 0
    calc_iterations = 0

    batch_iterator = tqdm(data, mininterval=2, desc='  - (Training)   ', leave=False)
    with torch.set_grad_enabled(is_train):
        for batch in batch_iterator:
            if is_train: optimizer.zero_grad()
            
            x, y = batch['image'].to(device), batch['attributes'].to(device)
            preds = model(x)
            # do_calculations = (iterations % calculate_every == 0) or not is_train
            loss, n_correct, total_labels, f1 = u.calculate_performance(preds, y, do_calculations=True)

            if is_train:
                loss.backward()
                optimizer.step()
            
            total_loss += loss.item()
            iterations += 1
            # if do_calculations:
            n_total_correct += n_correct
            n_total += total_labels
            total_f1 += f1
            calc_iterations += 1

            batch_iterator.set_description( 'accuracy: {accu:3.3f}%, avg_loss: {avg_loss:8.5f}, avg_f1: {f1:3.3f}'.format(accu=100*n_total_correct/n_total,
                avg_loss=total_loss/iterations, f1=total_f1/calc_iterations))
    
    return total_loss, n_total_correct / n_total, iterations, total_f1 / calc_iterations

In [6]:
start = time.time()
valid_loss, valid_accu, valid_count, valid_avg_f1 = run_epoch(xception_model, data, device, is_train=False)
print('  - (Xception) accuracy: {accu:3.3f} %, '\
        'avg_loss: {loss:8.5f}, avg_f1: {f1:3.3f}, elapse: {elapse:3.3f} min'.format(
            accu=100*valid_accu, loss=valid_loss/valid_count, f1=valid_avg_f1, elapse=(time.time()-start)/60))

  - (Training)   :   0%|          | 0/7 [00:00<?, ?it/s]/opt/anaconda3/envs/cs231n/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/envs/cs231n/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
                                                                                                   

  - (Xception) accuracy: 99.608 %, avg_loss:  0.00100, avg_f1: 0.982, elapse: 0.133 min


In [2]:
mobile_image_input_size = 224
dataset_root = '../../data/largescale/'
device = torch.device('cuda')
normalize_mobile = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) 
test_mobile = pickle.load( open(  os.path.join(dataset_root,"test.pkl"), "rb" ) )
test_mobile.transform = transforms.Compose([ transforms.CenterCrop(mobile_image_input_size), transforms.ToTensor(), normalize_mobile ])

In [3]:
mobile_data = d.DataLoader(test_mobile, batch_size=32, shuffle=True, drop_last=False, num_workers=2, pin_memory=True)

mobilenet_model = AttributeClassifierMobileNet(out_features=337, device=device)
baseline_checkpoint = torch.load('../trained_models/baseline_mobilenet.chkpt')
mobilenet_model.load_state_dict(baseline_checkpoint['model'])
mobilenet_model.eval()

AttributeClassifierMobileNet(
  (bm): MobileNetV2(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
          (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, 

In [6]:
start = time.time()
valid_loss, valid_accu, valid_count, valid_avg_f1 = run_epoch(mobilenet_model, mobile_data, device, is_train=False)
print('  - (Mobilenet) accuracy: {accu:3.3f} %, '\
        'avg_loss: {loss:8.5f}, avg_f1: {f1:3.3f}, elapse: {elapse:3.3f} min'.format(
            accu=100*valid_accu, loss=valid_loss/valid_count, f1=valid_avg_f1, elapse=(time.time()-start)/60))

  - (Training)   :   0%|          | 0/7 [00:00<?, ?it/s]/opt/anaconda3/envs/cs231n/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/envs/cs231n/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
                                                                                                   

  - (Mobilenet) accuracy: 97.337 %, avg_loss:  0.00890, avg_f1: 0.827, elapse: 0.080 min
